# Problema de Busca - Super Mario Bros
## Projeto da disciplina **SCC0230 - Inteligência Artificial**
### Busca Informada e Busca Cega

## Membros

* Alexandre Eduardo de Souza Jesus - alexandre_souza@usp.br -

* Arthur Santorum Lorenzetto - arts.lorenzetto@usp.br -

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - (Vice-Líder)

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472 (Líder)

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 

* Vitor Okubo Ianella - vitorok.03@gmail.com - 

## Introdução

* Motivação = Descrever o objetivo do trabalho;
* Especificação = Explicar a temática do problema e ferramentas utilizadas;
* Contextualização = Resumir as estratégias abordadas.

## Modelagem do Problema

Descrever os seguintes aspectos do problema:
* Estados;
* Operadores;
* Movimentos válidos;
* Configuração final.

Detalhar visualmente o Espaço de Estados (Matriz Esparsa)

## Busca Não Informada

* Justificar escolha do algoritmo;
* Especificar decisões de implementação: Estruturas de Dados, manipulação dos dados, adaptações ao problema, etc.

In [1]:
# Implementar a Busca Cega do Trabalho (Busca em Profundidade ou em Largura)
# Em Largura: https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
# Em Profundidade: https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=lbp

## Busca Informada

* Justificar escolha do algoritmo;
* Fundamentar heurística utilizada;
* Especificar decisões de implementação: Estruturas de Dados, manipulação dos dados, adaptações ao problema, etc.

In [2]:
# Implementar a Busca Informada do Trabalho
# A*: https://www.geeksforgeeks.org/a-search-algorithm/
# Greedy Best-First: https://www.geeksforgeeks.org/best-first-search-informed-search/


## Conclusão

* Comparar algoritmos com alguns casos de Teste;
* Interpretar resultados obtidos;
* Considerações Finais acerca do problema.

## Referências


BRATKO, I. Prolog Programming for Artificial Intelligence . Mitchell, T. Machine
Learning , McGraw Hill, 1997.

https://www.geeksforgeeks.org/difference-between-informed-and-uninformed-search-in-ai/?ref=lbp

https://www.sciencedirect.com/topics/mathematics/manhattan-distance

https://www.geeksforgeeks.org/difference-between-bfs-and-dfs/?ref=lbp

http://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html

http://theory.stanford.edu/~amitp/GameProgramming/